In [7]:
import openeo
import xarray
import matplotlib.pyplot as plt
import numpy as np

In [8]:
conn = openeo.connect("openeo.dataspace.copernicus.eu")
conn.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

In [11]:
import math
def FindOffsetLatLongFromDistance(lat, lon, distance):

    #Earth’s radius, sphere
    R=6378137

    #offsets in meters
    dn = distance
    de = distance

    #Coordinate offsets in radians
    dLat = dn/R
    dLon = de/(R*math.cos(math.pi*lat/180))

    #OffsetPosition, decimal degrees
    latO = lat + dLat * 180/math.pi
    lonO = lon + dLon * 180/math.pi 
    return latO, lonO

In [12]:
conn.describe_collection("SENTINEL3_OLCI_L1B")
left = -58.69330372108722
top = -8.433467471613604
bottom, right = FindOffsetLatLongFromDistance(-58.69330372108722, -8.433467471613604, 10)
bbox = {"west": left, "south": bottom, "east": right, "north": top}
sentinel3 = conn.load_collection(
    "SENTINEL3_OLCI_L1B",
    spatial_extent=bbox,
    bands=["B04", "B05", "B06", "B07"],
)

In [13]:
sentinel3.download("sentinel3_amazon.nc")

KeyboardInterrupt: 

In [ ]:
ds = xarray.load_dataset("sentinel3.nc")

# Convert xarray DataSet to a (bands, t, x, y) DataArray
data = ds[["B04", "B05", "B06", "B07", "B09", "B14"]].to_array(dim="bands")
fig, (axrgb, axhist) = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

# plot the data
data[{"t": 0}].plot.imshow(ax=axrgb)

# Plot the data histogram
data.plot.hist(bins=50, ax=axhist, histtype="stepfilled")

plt.show()